# Import

In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import norm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error

from google.colab import drive
drive.mount('/content/drive')

# Loading and processing Data

In [ ]:
# Loading predictions and corresponding actuals
actuals = pd.read_csv("/content/drive/MyDrive/Thesis/Models/Predictions/AR(1)Actuals.csv", sep=',')
AR_1 = pd.read_csv("/content/drive/MyDrive/Thesis/Models/Predictions/AR(1)Predictions.csv", sep=',')
AR_optimal = pd.read_csv("/content/drive/MyDrive/Thesis/Models/Predictions/AR(Optimal)Predictions.csv", sep=',')
ARIMA = pd.read_csv("/content/drive/MyDrive/Thesis/Models/Predictions/ARIMAPredictions.csv", sep=',')
RandomForest = pd.read_csv("/content/drive/MyDrive/Thesis/Models/Predictions/RandomForestPredictions.csv", sep=',')
XGBoost = pd.read_csv("/content/drive/MyDrive/Thesis/Models/Predictions/XGBoostPredictions.csv", sep=',')

In [ ]:
# Renaming the date column to date
actuals.rename(columns={'PredictionDate': 'Date'}, inplace=True)
AR_1.rename(columns={'PredictionDate': 'Date'}, inplace=True)
AR_optimal.rename(columns={'PredictionDate': 'Date'}, inplace=True)
ARIMA.rename(columns={'PredictionDate': 'Date'}, inplace=True)

RandomForest.rename(columns={'OriginalIndex': 'Date'}, inplace=True)
XGBoost.rename(columns={'OriginalIndex': 'Date'}, inplace=True)

RandomForest.rename(columns={'State': 'state'}, inplace=True)
XGBoost.rename(columns={'State': 'state'}, inplace=True)

In [ ]:
# Making Combination forecasts
combined_df1 = pd.concat([AR_optimal, ARIMA, RandomForest, XGBoost], ignore_index=True)
averaged_predictions1 = combined_df1.groupby(['Date', 'state']).mean().reset_index()
Combination_AR_1 = averaged_predictions1.sort_values(by=['state', 'Date']).reset_index(drop=True)

In [ ]:
# Removing 'USA' from the predictions and actuals
actuals = actuals[actuals['state'] != 'USA']
AR_1 = AR_1[AR_1['state'] != 'USA']
AR_optimal = AR_optimal[AR_optimal['state'] != 'USA']
ARIMA = ARIMA[ARIMA['state'] != 'USA']
RandomForest = RandomForest[RandomForest['state'] != 'USA']
XGBoost = XGBoost[XGBoost['state'] != 'USA']
Combination_AR_1 = Combination_AR_1[Combination_AR_1['state'] != 'USA']

In [ ]:
# Setting the dataframes to datetime
actuals['Date'] = pd.to_datetime(actuals['Date'])
AR_1['Date'] = pd.to_datetime(AR_1['Date'])
AR_optimal['Date'] = pd.to_datetime(AR_optimal['Date'])
ARIMA['Date'] = pd.to_datetime(ARIMA['Date'])
RandomForest['Date'] = pd.to_datetime(RandomForest['Date'])
XGBoost['Date'] = pd.to_datetime(XGBoost['Date'])
Combination_AR_1['Date'] = pd.to_datetime(Combination_AR_1['Date'])

# Performing the DM test for entire out-of-sample period

In [ ]:
# Setting the dataframes to have the same length
actuals = actuals[actuals['Date'] >= pd.Timestamp('2008-07-01')].reset_index(drop=True)
AR_1 = AR_1[AR_1['Date'] >= pd.Timestamp('2008-07-01')].reset_index(drop=True)
AR_optimal = AR_optimal[AR_optimal['Date'] >= pd.Timestamp('2008-07-01')].reset_index(drop=True)
ARIMA = ARIMA[ARIMA['Date'] >= pd.Timestamp('2008-07-01')].reset_index(drop=True)
RandomForest = RandomForest[RandomForest['Date'] >= pd.Timestamp('2008-07-01')].reset_index(drop=True)
XGBoost = XGBoost[XGBoost['Date'] >= pd.Timestamp('2008-07-01')].reset_index(drop=True)
Combination_AR_1 = Combination_AR_1[Combination_AR_1['Date'] >= pd.Timestamp('2008-07-01')].reset_index(drop=True)

In [ ]:
# List of actuals and models used in performing the DM test
actuals1 = actuals
models = {
    'AR_1': AR_1,
    'XGBoost': XGBoost,
    'RandomForest': RandomForest,
    'AR_optimal': AR_optimal,
    'ARIMA': ARIMA,
    'Combination': Combination_AR_1
}

In [ ]:
horizons = ['h1', 'h3', 'h6', 'h12']
states = actuals['state'].unique()

# Handling data on state and horizon key
errors = {state: {h: {} for h in horizons} for state in states}

# Calculating errors based on actuals and the respective model
for state in states:
    for model_name, df_model in models.items():
        for h in horizons:
            actuals_state = actuals1[actuals1['state'] == state]
            df_merged = pd.merge(actuals_state[['state', 'Date', h]], df_model[['state', 'Date', h]], on=['state', 'Date'], how='inner', suffixes=('_act', '_pred'))
            errors[state][h][model_name] = df_merged[f'{h}_act'] - df_merged[f'{h}_pred']

In [ ]:
# Diebold-Mariano test

def Diebold_Mariano_Test(model_errors,benchmark_erros, h):
    LossFunc = np.array(benchmark_erros.dropna())**2 - np.array(model_errors.dropna())**2
    maxLag = h
    DM = pd.DataFrame({'y':LossFunc})
    reg = smf.ols('y~1', data=DM).fit(cov_type='HAC',cov_kwds={'maxlags':np.max(maxLag),'kernel':'bartlett'})
    intercept = reg.params['Intercept']
    test_statistic = reg.tvalues['Intercept']
    p_Value = 1-norm.cdf(test_statistic)

    return p_Value, intercept, test_statistic

In [ ]:
states = actuals['state'].unique()
horizons = [1, 3, 6, 12]
Benchmark = 'AR_1'

# Placeholder for DM test results
DM_results = pd.DataFrame(columns=['State', 'Horizon', 'Model', 'P_Value', 'DM_Statistic', 'Params'])

# Performing the Diebold-Mariano test excluding the benchmark in the results dataframe
for state in states:
    for h in horizons:
        for model_name in models.keys():
            if model_name != Benchmark:
                p_Value, intercept, test_statistic = Diebold_Mariano_Test(errors[state][f'h{h}'][model_name], errors[state][f'h{h}'][Benchmark], h=h)
                # Creating a new df with results
                new_row = pd.DataFrame({
                    'State': [state],
                    'Horizon': [f'h{h}'],
                    'Model': [model_name],
                    'P_Value': [p_Value],
                    'DM_Statistic': [test_statistic],
                    'Params': [intercept]
                })
                # Concat results into one df
                DM_results = pd.concat([DM_results, new_row], ignore_index=True)

In [ ]:
DM_results.to_excel("/content/drive/MyDrive/Thesis/Performance Calculations/DieboldMarianoTestResults.xlsx")

# Performing the DM test for out-of-sample period excluding crisis period

In [ ]:
# Excluding crisis
actuals = actuals[(actuals['Date'] >= pd.Timestamp('2011-01-01')) & (actuals['Date'] <= pd.Timestamp('2019-12-01'))].reset_index(drop=True)
AR_1 = AR_1[(AR_1['Date'] >= pd.Timestamp('2011-01-01')) & (AR_1['Date'] <= pd.Timestamp('2019-12-01'))].reset_index(drop=True)
AR_optimal = AR_optimal[(AR_optimal['Date'] >= pd.Timestamp('2011-01-01')) & (AR_optimal['Date'] <= pd.Timestamp('2019-12-01'))].reset_index(drop=True)
ARIMA = ARIMA[(ARIMA['Date'] >= pd.Timestamp('2011-01-01')) & (ARIMA['Date'] <= pd.Timestamp('2019-12-01'))].reset_index(drop=True)
XGBoost = XGBoost[(XGBoost['Date'] >= pd.Timestamp('2011-01-01')) & (XGBoost['Date'] <= pd.Timestamp('2019-12-01'))].reset_index(drop=True)
RandomForest = RandomForest[(RandomForest['Date'] >= pd.Timestamp('2011-01-01')) & (RandomForest['Date'] <= pd.Timestamp('2019-12-01'))].reset_index(drop=True)
Combination_AR_1 = Combination_AR_1[(Combination_AR_1['Date'] >= pd.Timestamp('2011-01-01')) & (Combination_AR_1['Date'] <= pd.Timestamp('2019-12-01'))].reset_index(drop=True)

In [ ]:
# List of actuals and models used in performing the DM test
actuals1 = actuals
models = {
    'AR_1': AR_1,
    'XGBoost': XGBoost,
    'RandomForest': RandomForest,
    'AR_optimal': AR_optimal,
    'ARIMA': ARIMA,
    'Combination': Combination_AR_1
}

In [ ]:
horizons = ['h1', 'h3', 'h6', 'h12']
states = actuals['state'].unique()

# Handling data on state and horizon key
errors = {state: {h: {} for h in horizons} for state in states}

# Calculating errors based on actuals and the respective model
for state in states:
    for model_name, df_model in models.items():
        for h in horizons:
            actuals_state = actuals1[actuals1['state'] == state]
            df_merged = pd.merge(actuals_state[['state', 'Date', h]], df_model[['state', 'Date', h]], on=['state', 'Date'], how='inner', suffixes=('_act', '_pred'))
            errors[state][h][model_name] = df_merged[f'{h}_act'] - df_merged[f'{h}_pred']

In [ ]:
# Diebold-Mariano test

def Diebold_Mariano_Test(model_errors,benchmark_erros, h):
    LossFunc = np.array(benchmark_erros.dropna())**2 - np.array(model_errors.dropna())**2
    maxLag = h
    DM = pd.DataFrame({'y':LossFunc})
    reg = smf.ols('y~1', data=DM).fit(cov_type='HAC',cov_kwds={'maxlags':np.max(maxLag),'kernel':'bartlett'})
    intercept = reg.params['Intercept']
    test_statistic = reg.tvalues['Intercept']
    p_Value = 1-norm.cdf(test_statistic)

    return p_Value, intercept, test_statistic

In [ ]:
states = actuals['state'].unique()
horizons = [1, 3, 6, 12]
Benchmark = 'AR_1'

# Placeholder for DM test results
DM_results_excl_crisis = pd.DataFrame(columns=['State', 'Horizon', 'Model', 'P_Value', 'DM_Statistic', 'Params'])

# Performing the Diebold-Mariano test excluding the benchmark in the results dataframe
for state in states:
    for h in horizons:
        for model_name in models.keys():
            if model_name != Benchmark:
                p_Value, intercept, test_statistic = Diebold_Mariano_Test(errors[state][f'h{h}'][model_name], errors[state][f'h{h}'][Benchmark], h=h)
                # Creating a new df with results
                new_row = pd.DataFrame({
                    'State': [state],
                    'Horizon': [f'h{h}'],
                    'Model': [model_name],
                    'P_Value': [p_Value],
                    'DM_Statistic': [test_statistic],
                    'Params': [intercept]
                })
                # Concat results into one df
                DM_results_excl_crisis = pd.concat([DM_results_excl_crisis, new_row], ignore_index=True)

In [ ]:
DM_results_excl_crisis.to_excel("/content/drive/MyDrive/Thesis/Performance Calculations/ExclCrisisDieboldMarianoTestResults.xlsx")

In [ ]:
# END